In [41]:
import os
import pandas as pd
import ast # string parse

# Dataset path
data_dir = "/Users/tuboshu/Downloads/Polar_Dataset"  
annotation_file = "polar_annotations.xlsx"  

df_train = pd.read_excel(os.path.join(data_dir, annotation_file), sheet_name="train_set")
df_val = pd.read_excel(os.path.join(data_dir, annotation_file), sheet_name="validation_set")

In [42]:
# process train file
print(df_train.head)

# 3 classes, only use adenoma and hyperplastic polyp in our project
polyp_classes = df_train["PATHOLOGY DIAGNOSIS"].unique()
print(polyp_classes)

category_counts = df_train["PATHOLOGY DIAGNOSIS"].value_counts()
print(category_counts)


<bound method NDFrame.head of                             PATIENT_ID                          POLYP_ID  \
0     1738b15b7fb1cd2d88e4c6db8ecc105a  10fc3a817b8f1c79cd3b97f17bffe141   
1     1738b15b7fb1cd2d88e4c6db8ecc105a  10fc3a817b8f1c79cd3b97f17bffe141   
2     f10b7063a7d05c9287eee831c543e172  8f10594a9127b19e202461b554eabed2   
3     f10b7063a7d05c9287eee831c543e172  8f10594a9127b19e202461b554eabed2   
4     9304f26a2bc220cf98a79e2b8b3ab798  03ddd338799540d5ffbc52c5d8fb4b1f   
...                                ...                               ...   
2632  55dac536cf9f10af9299eed3deb64efb  c3434f07e2d52c50333ee2c5bec3cea2   
2633  55dac536cf9f10af9299eed3deb64efb  c3434f07e2d52c50333ee2c5bec3cea2   
2634  55dac536cf9f10af9299eed3deb64efb  c3434f07e2d52c50333ee2c5bec3cea2   
2635  e07730acabe96d3f9a8752388081fcf6  fddaa3d77a75d3cf5531ccf016596ea8   
2636  e07730acabe96d3f9a8752388081fcf6  fddaa3d77a75d3cf5531ccf016596ea8   

                           IMG_FILE_PREFIX POLYP POSITION

In [43]:
row = df_train.iloc[2]
print(str(row['PATIENT_ID'])) 
print(str(row['POLYP_ID']))
print(str(row['IMG_FILE_PREFIX'])) 
print(row["POLYP POSITION (x,y,w,h)"])
print(row["SIZE"])
print(row['PATHOLOGY DIAGNOSIS'])

def parse_bbox(bbox_str):
    try:
        bbox = ast.literal_eval(bbox_str)  
        if isinstance(bbox, list) and len(bbox) == 4:
            return bbox  
    except:
        pass
    return None  
print(parse_bbox(row["POLYP POSITION (x,y,w,h)"]))
print(parse_bbox(row["POLYP POSITION (x,y,w,h)"])[2])

f10b7063a7d05c9287eee831c543e172
8f10594a9127b19e202461b554eabed2
865f0321-37a5-4347-9756-b6fbdf29001b
[330, 77, 193, 176]
2
adenoma
[330, 77, 193, 176]
193


In [44]:
# Filter 2 polyps and save the train/valid split info into a new file
target_labels = ["adenoma", "hyperplastic polyp"]
data_list = []

# Loop through rows, decide each row belongs to train/valid set, and calculate bbox area
for index, row in df_train.iterrows():
    patient_id = str(row['PATIENT_ID']) 
    polyp_id = str(row['POLYP_ID']) 
    image_prefix = str(row['IMG_FILE_PREFIX'])  
    polyp_position = row["POLYP POSITION (x,y,w,h)"]
    size = row["SIZE"]
    label = row['PATHOLOGY DIAGNOSIS']  
    list_polyp_position = parse_bbox(polyp_position)
    bbox_area = int(list_polyp_position[2] * list_polyp_position[3])
    if label in target_labels:
        train_img_folder = os.path.join(data_dir, "train_set", patient_id, polyp_id)#, image_prefix + '.full.png')
        if os.path.exists(train_img_folder):
            for img_file in os.listdir(train_img_folder):
                if img_file.startswith(image_prefix): 
                    img_path = os.path.join(train_img_folder, img_file)
                    data_list.append([patient_id, polyp_id, image_prefix, polyp_position, size, label, bbox_area, "train"])  

        else:
            print(f"Folder {train_img_folder} don't exist!")
            print(f"Details:\n patient_id: {patient_id} \n polyp_id: {polyp_id} \n image_prefix: {image_prefix}\n label: {label}\n")


df_filterd = pd.DataFrame(data_list, columns=["PATIENT_ID", "POLYP_ID", "IMG_FILE_PREFIX", "POLYP POSITION (x,y,w,h)", "SIZE", "PATHOLOGY DIAGNOSIS", "BBOX_AREA", "SPLIT"])
print(df_filterd.head)
output_csv_path = "polar_dataset_filterd.csv"
#df_filterd.to_csv(output_csv_path, index=False, mode='w', header=True) # overwrite previous one
df_filterd.to_csv(output_csv_path, index=False)

Folder /Users/tuboshu/Downloads/Polar_Dataset/train_set/71ab9f248af73c91f6310a91e967a11d/030ae2b25c1117f250137d2dd247382f don't exist!
Details:
 patient_id: 71ab9f248af73c91f6310a91e967a11d 
 polyp_id: 030ae2b25c1117f250137d2dd247382f 
 image_prefix: 784820c6-58a5-4814-9795-41d99db2ab53
 label: adenoma

<bound method NDFrame.head of                             PATIENT_ID                          POLYP_ID  \
0     f10b7063a7d05c9287eee831c543e172  8f10594a9127b19e202461b554eabed2   
1     f10b7063a7d05c9287eee831c543e172  8f10594a9127b19e202461b554eabed2   
2     2092490df624877b6963f7a0afb7ba76  c48bb221b3559bdf5726b1c269996161   
3     2092490df624877b6963f7a0afb7ba76  c48bb221b3559bdf5726b1c269996161   
4     2092490df624877b6963f7a0afb7ba76  5524a665e549a32e3b581180ff34a71d   
...                                ...                               ...   
2370  55dac536cf9f10af9299eed3deb64efb  c3434f07e2d52c50333ee2c5bec3cea2   
2371  55dac536cf9f10af9299eed3deb64efb  c3434f07e2d52c503

In [48]:
# process val file
print(df_val.head)

<bound method NDFrame.head of                            PATIENT_ID                        POLYP/IMAGE_ID  \
0    287a18ae061a4f779a02d787f01e823c  57781dbd-5c47-4e9f-985e-62a9077ea9c5   
1    287a18ae061a4f779a02d787f01e823c  ddb3f8d6-1f22-4576-8f74-d9d4751a48da   
2    287a18ae061a4f779a02d787f01e823c  64319dc0-e72d-4902-8084-ecdc17136bdd   
3    287a18ae061a4f779a02d787f01e823c  bc673cdc-2e3a-41c7-8b37-cf289a3e42d5   
4    287a18ae061a4f779a02d787f01e823c  8f155836-88df-4521-980e-5145a183879c   
..                                ...                                   ...   
725  ac64958c6cf14c86bac2af0276564483  4507f610-6f19-4fe5-aa6b-2723d8b30b0a   
726  ddedb4c18dbf4a66835737b713e1e829  1e1b271d-ed88-46dd-b336-758e075da790   
727  bf7cd27d2c55460f96fec721b409f8f8  dd2fb47c-dd47-4286-a39c-28bbd9e66ea9   
728  eee29ab4dc6048029aefab30c3861507  c43e5751-cc59-43a3-b8f4-ce357acdfc37   
729  dc41827510ea4f878f48b2088596bd61  604813dd-179f-47ae-bacf-ddee3b619baa   

    POLYP POSITION (x

In [52]:
# 12 classes, only use adenoma and hyperplastic polyp in our project
polyp_classes = df_val["PATHOLOGY DIAGNOSIS"].unique()
print(polyp_classes)

category_counts = df_val["PATHOLOGY DIAGNOSIS"].value_counts()
print(category_counts)

adenoma_count = df_val[df_val["PATHOLOGY DIAGNOSIS"].str.contains("adenoma", case=False, na=False)]["PATHOLOGY DIAGNOSIS"].value_counts().sum()

print(f"\nTotal Adenoma Cases: {adenoma_count}")

['Tubular adenoma' 'Sessile serrated lesion' 'Normal mucosa'
 'Villous adenoma' 'Tubulovillous adenoma' 'Hyperplastic'
 'No polyp received' 'Other...' 'Adenocarcinoma'
 'Traditional serrated adenoma' nan '##USER_MISSING_99##']
PATHOLOGY DIAGNOSIS
Tubular adenoma                 426
Hyperplastic                     99
Sessile serrated lesion          63
Tubulovillous adenoma            44
No polyp received                31
Normal mucosa                    26
Other...                         15
Adenocarcinoma                   11
Traditional serrated adenoma      5
Villous adenoma                   3
##USER_MISSING_99##               1
Name: count, dtype: int64

Total Adenoma Cases: 478


In [50]:
polyp_classes = df_val["OPTICAL DIAGNOSIS"].unique()
print(polyp_classes)

category_counts = df_val["OPTICAL DIAGNOSIS"].value_counts()
print(category_counts)

['Adenoma' 'Sessile serrated lesion' 'Hyperplastic polyp'
 '##USER_MISSING_99##' 'Other' 'Carcinoma']
OPTICAL DIAGNOSIS
Adenoma                    510
Sessile serrated lesion    107
Hyperplastic polyp          91
Carcinoma                   12
Other                        7
##USER_MISSING_99##          3
Name: count, dtype: int64


In [ ]:
target_labels = ["adenoma", "hyperplastic polyp"]
data_list = []

# Further steps to analyse...

1. Statistical category distribution (diff polyps in train/val set)

2. Check the image size to determine whether Resize is required (224,224)

3. Check the Image Brightness & Color Distribution

4. Preprocessing. normalization? data augmentation?
